In [1]:
%pylab inline
%config InlineBackend.print_figure_kwargs={'facecolor' : "w"}

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [7]:
from nbodykit.lab import BigFileCatalog
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
from hmf import MassFunction  
from nbodykit.transform import ConcatenateSources, CartesianToEquatorial
import healpy as hp
from astropy.cosmology import FlatLambdaCDM
from nbodykit.lab import FFTPower, FFTCorr 
import illustris_python as il
from nbodykit.lab import ArrayCatalog
from scipy import interpolate
import healpy as hp
from classy import Class
from scipy.interpolate import interp1d
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd

In [8]:
# your existing font dict is fine if you want; this is the key bit:
mpl.rcParams['mathtext.fontset'] = 'cm'          # use Computer Modern math
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = ['CMU Serif', 'Computer Modern Roman', 'DejaVu Serif']  # fallbacks


In [9]:
import os
class cd:
    """Context manager for changing the current working directory"""
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [10]:
"""cosmology with certain cosmological parameters, in our case, we set h=0.6774, omega0_m=0.3089"""
##### LCDM parameters
h=0.6774
Om0 = 0.309
H0=h*100

cosmo = FlatLambdaCDM(H0=H0, Om0=Om0)

In [11]:
"""Read all 10 realizations of Halfdome"""
import glob
from os.path import join
path_data = '/global/cfs/cdirs/cmb/data/halfdome/stampede2_3750Mpch_6144cube/final_res'
cat_lastname = '3750Mpc_2048node_seed_'
folder_lastname = '/rfof_proc131072_nc6144_size3750_'
cluster_var = [x for x in glob.glob(join(path_data,cat_lastname+'*'+folder_lastname+'*'))]
cluster_var.sort()
print(cluster_var)

[]


In [25]:
"""info of outputs"""
partl = BigFileCatalog(cluster_var[0]+'/usmesh/', dataset='1')
halos = BigFileCatalog(cluster_var[0]+'/usmesh/', dataset='RFOF')
sheet = BigFileCatalog(cluster_var[0]+'/usmesh/', dataset='HEALPIX')
sheet.attrs

{'BoxSize': array([3750.]),
 'GrowthFactor': array([0.12745536]),
 'GrowthRate': array([0.99878173]),
 'HubbleE': array([17.59520105]),
 'HubbleParam': array([0.6774]),
 'LibFastPMVersion': array(['1', '.', '0', '.', '7', '5', 'd', '8', '5', 'c', '1', 'd', '5',
        '1'], dtype='<U1'),
 'MassTable': array([0.       , 1.9487254, 0.       , 0.       , 0.       , 0.       ]),
 'NC': array([6144]),
 'Omega0': array([0.3089]),
 'OmegaLambda': array([0.6911]),
 'OmegaM': array([0.3089]),
 'Omega_cdm': array([0.3089]),
 'ParamFile': array(['{', '\n', '\t', ..., '\n', '}', ''], dtype='<U1'),
 'ParticleFraction': array([1.]),
 'RSDFactor': array([0.00568337]),
 'ScalingFactor': array([0.1]),
 'Time': array([0.1]),
 'TotNumPart': array([           0, 231928233984,            0,            0,
                   0,            0]),
 'UnitLength_in_cm': array([3.085678e+24]),
 'UnitMass_in_g': array([1.989e+43]),
 'UnitVelocity_in_cm_per_s': array([100000.]),
 'UsePeculiarVelocity': array([1], dt

In [13]:
def read_range(cat, amin, amax):
    """ Read a portion of the lightcone between two red shift ranges
        The lightcone from FastPM is sorted in Aemit and an index is built.
        So we make use of that.
        CrowCanyon is z > 0; We paste the mirror image to form a full sky.
    """
    edges = cat.attrs['aemitIndex.edges']
    offsets = cat.attrs['aemitIndex.offset']
    start, end = edges.searchsorted([amin, amax])
    if cat.comm.rank == 0:
        cat.logger.info("Range of index is %d to %d" %(( start + 1, end + 1)))
    start = offsets[start + 1]
    end = offsets[end + 1]
    cat =  cat.query_range(start, end)
    #cat1 = cat.copy()
    #cat1['Position'] = cat1['Position'] * [1, 1, -1.]
    #cat3 = ConcatenateSources(cat, cat1)
    if cat.csize > 0:
        cat['RA'], cat['DEC'] = CartesianToEquatorial(cat['Position'], frame='galactic')
    else:
        cat['RA'] = 0
        cat['DEC'] = 0
    return cat

In [10]:
"""Read mass sheet from lightcone output"""
def read_slice(cat, islice):
    """returning
    a1
    a2
    map_for_slice: density map
    overden: overdensity map"""
    aemitIndex_edges = cat.attrs['aemitIndex.edges']
    aemitIndex_offset = cat.attrs['aemitIndex.offset']
    npix = cat.attrs['healpix.npix'][0]
    nc = cat.attrs['NC'][0]
    boxsize = cat.attrs['BoxSize'][0]
    M_cdm = cat.attrs['MassTable'][1]
    rhobar = M_cdm * (nc / boxsize)**3
    
    a1 = aemitIndex_edges[islice]
    a2 = aemitIndex_edges[islice + 1]
    start = aemitIndex_offset[1 + islice]
    stop = aemitIndex_offset[2 + islice]
    z1 = 1. / a1 - 1.
    z2 = 1. / a2 - 1.
    d1 = cosmo.comoving_distance(z1).value * h
    d2 = cosmo.comoving_distance(z2).value * h
    pid = cat['ID'][start:stop].compute()
    mass = cat['Mass'][start:stop].compute()
    ipix = pid % npix
    islice = ipix // npix
    map_for_slice = np.bincount(ipix, weights=mass, minlength=npix)
    volume_diff = 4 * np.pi * (d1**3 - d2**3) / (3 * npix)
    rho = map_for_slice / volume_diff 
    overden = rho / rhobar 
    return a1, a2, map_for_slice, overden

In [12]:
def velsheet(cat, islice):
    """returning
    a1
    a2
    map_for_slice: velocity map
    deltavr_plus1: velocity map/averaged density"""
    aemitIndex_edges = cat.attrs['aemitIndex.edges']
    aemitIndex_offset = cat.attrs['aemitIndex.offset']
    npix = cat.attrs['healpix.npix'][0]
    nc = cat.attrs['NC'][0]
    boxsize = cat.attrs['BoxSize'][0]
    M_cdm = cat.attrs['MassTable'][1]
    rhobar = M_cdm * (nc / boxsize)**3
    
    a1 = aemitIndex_edges[islice]
    a2 = aemitIndex_edges[islice + 1]
    start = aemitIndex_offset[1 + islice]
    stop = aemitIndex_offset[2 + islice]
    z1 = 1. / a1 - 1.
    z2 = 1. / a2 - 1.
    d1 = cosmo.comoving_distance(z1).value * h
    d2 = cosmo.comoving_distance(z2).value * h
    pid = cat['ID'][start:stop].compute()
    rmom = cat['Rmom'][start:stop].compute()
    mass = cat['Mass'][start:stop].compute()
    ipix = pid % npix
    islice = ipix // npix
    map_for_slice = np.bincount(ipix, weights=rmom/mass, minlength=npix) #radial velocity
    volume_diff = 4 * np.pi * (d1**3 - d2**3) / (3 * npix)
    vr = map_for_slice / volume_diff
    deltavr_plus1 = vr / rhobar
    return a1, a2, map_for_slice, deltavr_plus1

In [26]:
"""e.g., reading halo information from lightcone output at z=1 from seed=100 run"""
halo_1 = read_range(halos,0.49,0.51) #at z=1
halo_1

BigFileCatalog(size=82085937, FileStack(BigFile(path=/global/cfs/cdirs/cmb/data/halfdome/stampede2_3750Mpch_6144cube/final_res/3750Mpc_2048node_seed_100/rfof_proc131072_nc6144_size3750_nsteps60lin_ldr0_rcvfalse_fstnone_pnf2_lnf2_s100_dhf1.0000_tiled0.20_fll_elllim_10000_npix_8192_rfofkdt_8/usmesh, dataset=RFOF/, ncolumns=10, shape=(2854844506,)>, ... 1 files))

In [27]:
"""e.g., reading mass/velocity sheet information from lightcone output at z=1 from seed=100 run"""
nslice = 50
print('nside=',sheet.attrs['healpix.nside'][0])
a1, a2, map1, delta1  = read_slice(sheet, nslice)
a1, a2, vel_map1, vel_delta1  = velsheet(sheet, nslice)
z1 = 1. / a1 - 1.
z2 = 1. / a2 - 1.
z_s = (z1 + z2)/2
print(z_s)

nside= 8192
0.9803921568627451
